In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18192430
paper_name = 'linderholm_bisson_2008' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [12]:
original_data = pd.read_csv('raw_data/table3.txt', sep='\t')

In [13]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 40 x 2


In [14]:
original_data.head()

,Functiona,Gene(s)b
0,Purine biosynthesis,"ADE1, ADE2, ADE4, ADE5,7, ADE6, ADE8"
1,Stress response,RTS1
2,Tryptophan biosynthesis,"TRP3, TRP4, TRP5"
3,Amino acid metabolism,"GCV1, GCV2"
4,Protein metabolism,"DOA1, GCN1, UBP14"


In [15]:
original_data['genes'] = original_data['Gene(s)b'].apply(lambda x: x.split(', '))
genes = [x for _list in original_data['genes'] for x in _list ]

In [17]:
original_data = pd.DataFrame(data={'genes': genes, 'data': 1})

In [21]:
original_data2 = pd.read_csv('raw_data/white_colonies.txt', sep='\t', header=None, names=['genes','data'])
original_data2.head()

,genes,data
0,MET1,-1
1,MET5,-1
2,MET8,-1
3,MET10,-1


In [22]:
original_data = pd.concat([original_data, original_data2], axis=0)

In [28]:
original_data = original_data.reset_index()

In [29]:
original_data['genes'] = original_data['genes'].astype(str)

In [30]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [31]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['genes'], to='orf')

In [32]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [index_input, genes, data, orf]
Index: []


In [33]:
original_data.set_index('orf', inplace=True)

In [34]:
original_data = original_data[['data']].copy()

In [35]:
original_data = original_data.groupby(original_data.index).mean()

In [36]:
original_data.shape

(92, 1)

# Prepare the final dataset

In [37]:
data = original_data.copy()

In [38]:
dataset_ids = [16706]
datasets = datasets.reindex(index=dataset_ids)

In [39]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [40]:
data.head()

dataset_id,16706
data_type,value
orf,
YAR015W,1
YBR058C,1
YBR081C,1
YBR119W,1
YBR213W,-1


## Subset to the genes currently in SGD

In [41]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [42]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16706
,data_type,value
gene_id,orf,
67,YAR015W,1
254,YBR058C,1
276,YBR081C,1
313,YBR119W,1
407,YBR213W,-1


# Normalize

In [43]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [44]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [45]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      16706          
data_type       value    valuez
gene_id orf                    
67      YAR015W     1  7.028481
254     YBR058C     1  7.028481
276     YBR081C     1  7.028481
313     YBR119W     1  7.028481
407     YBR213W    -1 -6.887911

# Print out

In [46]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [47]:
from IO.save_data_to_db3 import *

In [48]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 18192430...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  3.49it/s]

Updating the data_modified_on field...
